In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from tfinterface.model_base import ModelBase
from tfinterface.reinforcement import ExperienceReplay
from tfinterface.utils import select_columns, soft_if, get_run
from phi.api import *
import tensorflow as tf
import random
from scipy.interpolate import interp1d
import numpy as np
import gym
from gym import wrappers
from tfinterface.reinforcement import ExpandedStateEnv
import os
import time


name = "actor-critic-reward-task"

In [60]:
class Inputs(object):
    def __init__(self, n_states, scope):
        with tf.variable_scope(scope):
            self.episode_length = tf.placeholder(tf.int64, [], name='episode_length')

            self.s = tf.placeholder(tf.float32, [None, n_states], name='s')
            self.a = tf.placeholder(tf.int32, [None], name='a')
            self.r = tf.placeholder(tf.float32, [None], name='r')
            self.v1 = tf.placeholder(tf.float32, [None], name='V1')
            self.done = tf.placeholder(tf.float32, [None], name='done')
            
            self.learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')
            self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
            self.training = tf.placeholder(tf.bool, [], name='training')
            
            self.pi = tf.placeholder(tf.float32, [], name='pi')
            

class Critic(object):
    def __init__(self, base_model, inputs, n_actions, n_states, y, reward_loss_proportion, scope):
        with tf.variable_scope(scope):
            
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            net = inputs.s

            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer", **ops)        
            
            self.V = tf.layers.dense(net, n_actions, name='V', **ops)[:, 0]
            
            r_net = net
#             r_net = tf.layers.dense(r_net, 32, name="r_relu_layer", activation=tf.nn.relu, **ops)
            r = tf.layers.dense(r_net, 1, name='r', **ops)[:, 0]
            
            
            self.r_loss = r - inputs.r |> tf.nn.l2_loss |> tf.reduce_mean

            self.target = soft_if(inputs.done, inputs.r,  inputs.r + y * inputs.v1)

            self.error = self.target - self.V
            self.loss = Pipe(self.error, tf.nn.l2_loss, tf.reduce_mean) + reward_loss_proportion * self.r_loss

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

            self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)

            avg_error, std_error = tf.nn.moments(self.error, [0])
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('avg_target', tf.reduce_mean(self.target)),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.scalar('avg_error', avg_error),
                tf.summary.scalar('std_error', std_error),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            
class Actor(object):
    def __init__(self, base_model, inputs, target_critic, n_actions, n_states, y, reward_loss_proportion, scope):
        with tf.variable_scope(scope):
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            net = inputs.s

            net = tf.layers.dense(net, 128, activation=tf.nn.relu, name="relu_layer", use_bias=True, **ops)
            net = tf.nn.dropout(net, inputs.keep_prob)

            self.P = tf.layers.dense(net, n_actions, activation=tf.nn.softmax, name='P', use_bias=False, **ops)
            self.Pa = select_columns(self.P, inputs.a)
            
            r_net = net
#             r_net = tf.layers.dense(r_net, 32, name="r_relu_layer", activation=tf.nn.relu, **ops)
            r = tf.layers.dense(r_net, 1, name='r', **ops)[:, 0]
            
            
            self.r_loss = r - inputs.r |> tf.nn.l2_loss |> tf.reduce_mean
            self.action_loss = - tf.log(tf.clip_by_value(self.Pa, 1e-3, 1.0)) * target_critic.error

            self.loss = self.action_loss + reward_loss_proportion * self.r_loss
            self.loss = tf.reduce_mean(self.loss)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

            self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)

            
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            


In [61]:
class LunarLander(ModelBase):
    
    def define_model(self, n_actions, n_states, reward_loss_proportion=0.1, y=0.98, buffer_length=50000, pi=0.1):
        self.global_max = float('-inf')

        self.replay_buffer = ExperienceReplay(max_length=buffer_length)


        with self.graph.as_default(), tf.device("cpu:0"):

            self.inputs = Inputs(n_states, "inputs")

            self.critic = Critic(self, self.inputs, n_actions, n_states, y, reward_loss_proportion, "critic")
            self.target_critic = Critic(self, self.inputs, n_actions, n_states, y, reward_loss_proportion, "target_critic")
            self.actor = Actor(self, self.inputs, self.target_critic, n_actions, n_states, y, reward_loss_proportion, "actor")

            self.update = tf.group(self.critic.update, self.actor.update)

            self.episode_length_summary = tf.summary.scalar('episode_length', self.inputs.episode_length)

            self.summaries = tf.summary.merge([self.actor.summaries, self.critic.summaries, self.target_critic.summaries])

            self.update_target = tf.group(*[
                t.assign_add(pi * (a - t)) for t, a in zip(self.target_critic.variables, self.critic.variables)
            ])
    
    
    def predict_feed(self, S):
        return {
            self.inputs.s: S,
            self.inputs.keep_prob: 1.0,
            self.inputs.training: False
        }
    
    def predict(self, state, e = 0.0):
        predict_feed = self.predict_feed([state])
        actions = self.sess.run(self.actor.P, feed_dict=predict_feed)
        actions = actions[0]
        n = len(actions)

        if random.random() < e:
            return random.randint(0, n-1)
        else:
            return np.random.choice(n, p=actions)
    
    def fit_feed(self, S, A, R, V1, Done, learning_rate, keep_prob):
        return {
            self.inputs.s: S,
            self.inputs.a: A,
            self.inputs.r: R,
            self.inputs.v1: V1,
            self.inputs.done: Done,
            self.inputs.learning_rate: learning_rate,
            self.inputs.keep_prob: keep_prob,
            self.inputs.training: True
        }
    
    
    def fit(self, env, keep_prob=0.5, e=0.01, learning_rate=0.01, print_step=10, 
            update_target_step = 32, episodes=100000, max_episode_length=float('inf'), batch_size=32):
        
        r_total = 0.

        for episode in range(episodes):
            done = False
            ep_step = 0
            s = env.reset()
            episode_length = 0
            ep_reward = 0.
            
            while not done and ep_step <= max_episode_length:
                self.global_step += 1
                episode_length += 1
                ep_step += 1
                
                
                _learning_rate = learning_rate(self.global_step) if hasattr(learning_rate, '__call__') else learning_rate
                _e = e(self.global_step) if hasattr(e, '__call__') else e
                
                
                a = self.predict(s, e = _e)
                s1, r, done, info = env.step(a)
                r_total += r
                ep_reward += r
                
                
                self.replay_buffer.append((s, a, r, s1, float(done)))
                
                
                S, A, R, S1, Done = self.replay_buffer.random_batch(batch_size).unzip()
                predict_feed = self.predict_feed(S1)
                V1 = self.sess.run(self.target_critic.V, feed_dict=predict_feed)

                
                fit_feed = self.fit_feed(S, A, R, V1, Done, _learning_rate, keep_prob)
                _, summaries = self.sess.run([self.update, self.summaries], feed_dict=fit_feed)
                self.writer.add_summary(summaries, self.global_step)
                
                
                if self.global_step % update_target_step == 0:
                    self.sess.run(self.update_target)
                
                
                s = s1
                
            
            episode_length_summary = self.sess.run(self.episode_length_summary,
                                                   feed_dict={self.inputs.episode_length: episode_length})
            self.writer.add_summary(episode_length_summary, self.global_step)


            if ep_reward >= self.global_max:
                print("[MAX] Episode: {}, Length: {}, Reward: {}, buffer_len: {}".format(episode, episode_length, ep_reward, len(self.replay_buffer)))
                self.save(model_path = self.model_path + ".{score}".format(score = ep_reward))
                self.global_max = ep_reward


            if episode % print_step == 0 and episode > 0:
                avg_r = r_total / print_step
                actor_loss = self.sess.run(self.actor.loss, feed_dict=fit_feed)
                print("[NOR] Episode: {}, Length: {}, Avg Reward: {}, e: {}, Learning Rate: {}, buffer_len: {}".format(episode, episode_length, avg_r, _e, _learning_rate, len(self.replay_buffer)))
                print("Loss: {}".format(actor_loss))
                self.save()
                r_total = 0.


In [62]:
run = get_run()
env = gym.make("LunarLander-v2")
env = wrappers.Monitor(env, "monitor/{run}".format(run = run))
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/models/{run}".format(path = os.getcwd(), run = run)
logs_path = "{path}/logs/{run}".format(path = os.getcwd(), run = run)


model = LunarLander(
    n_actions, n_states, y=0.9999, 
    buffer_length=500000,
    model_path = model_path,
    logs_path = logs_path,
    restore = False,
    pi = 0.005,
    reward_loss_proportion = 0.05
)

[2017-03-20 01:16:54,713] Making new env: LunarLander-v2
[2017-03-20 01:16:54,715] DEPRECATION WARNING: env.spec.timestep_limit has been deprecated. Replace your call to `env.spec.timestep_limit` with `env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')`. This change was made 12/28/2016 and is included in version 0.7.0
[2017-03-20 01:16:54,716] Creating monitor directory monitor/21


In [63]:
k = 40000.
model.fit(
    env, print_step=10, 
    episodes=int(1e5), max_episode_length=10000, batch_size=32,
    learning_rate = 0.01, # lambda t: 0.05 * k / (k + t)
    e = interp1d([0, 300000], [0.4, 0.05], fill_value=0.05, bounds_error=False),
    keep_prob = 0.5,
    update_target_step = 1
)

[2017-03-20 01:16:55,743] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video000000.mp4
[2017-03-20 01:16:57,072] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video000001.mp4


[MAX] Episode: 0, Length: 84, Reward: -465.086305932, buffer_len: 84
[MAX] Episode: 2, Length: 71, Reward: -381.337579648, buffer_len: 227


[2017-03-20 01:17:00,102] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video000008.mp4


[NOR] Episode: 10, Length: 71, Avg Reward: -568.647667717, e: 0.398990833333, Learning Rate: 0.01, buffer_len: 866
Loss: 15.0958900452
[MAX] Episode: 16, Length: 62, Reward: -364.095665353, buffer_len: 1322
[MAX] Episode: 18, Length: 56, Reward: -324.584533853, buffer_len: 1464
[NOR] Episode: 20, Length: 53, Avg Reward: -487.224757581, e: 0.398164833333, Learning Rate: 0.01, buffer_len: 1574
Loss: 232.254516602


[2017-03-20 01:17:07,243] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video000027.mp4


...

[2017-03-20 10:08:25,640] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video035000.mp4


[NOR] Episode: 35000, Length: 132, Avg Reward: -88.7955336749, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 7.37106800079
[NOR] Episode: 35010, Length: 275, Avg Reward: -114.089726342, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 176.16784668
[NOR] Episode: 35020, Length: 123, Avg Reward: -111.307636352, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -1.04912829399
[NOR] Episode: 35030, Length: 207, Avg Reward: -89.649277904, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 2.23860836029
[NOR] Episode: 35040, Length: 118, Avg Reward: -98.7779294848, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 3.34817767143
[NOR] Episode: 35050, Length: 216, Avg Reward: -94.9367386668, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 11.0819587708
[NOR] Episode: 35060, Length: 205, Avg Reward: -78.1378979216, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -3.94248795509
[NOR] Episode: 35070, Length: 130, Avg Reward: -60.8689523812, e: 0.05, Lear

[2017-03-20 10:26:48,526] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/reward-task/monitor/21/openaigym.video.21.20184.video036000.mp4


[NOR] Episode: 36000, Length: 130, Avg Reward: -99.1670860506, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 9.17866325378
[NOR] Episode: 36010, Length: 176, Avg Reward: -25.3429787403, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 4.23073768616
[NOR] Episode: 36020, Length: 324, Avg Reward: 14.228374527, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 4.90224552155
[NOR] Episode: 36030, Length: 173, Avg Reward: -0.0574611458138, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 148.171020508
[NOR] Episode: 36040, Length: 363, Avg Reward: 36.2952381732, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 8.69233036041
[NOR] Episode: 36050, Length: 235, Avg Reward: -63.2415639042, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 13.37870121
[NOR] Episode: 36060, Length: 1000, Avg Reward: -26.4817633636, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 15.0110015869
[NOR] Episode: 36070, Length: 299, Avg Reward: 55.4505006636, e: 0.05, Learnin

KeyboardInterrupt: 

In [66]:

env = gym.make("LunarLander-v2")
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/models/21.272.691146069".format(path = os.getcwd(), name = name)
logs_path = "{path}/logs/".format(path = os.getcwd(), name = name)


model_run = LunarLander(
    n_actions, n_states,
    model_path = model_path,
    flush_secs = 3.0,
    restore = True
)

for i in range(100):
    s = env.reset()
    done = False
    total = 0.
    ep = 0
    while not done and ep < 700:
        ep += 1
        a = model_run.predict(s, 0.0)
        s, r, done, info = env.step(a)
        total += r
        env.render()
        time.sleep(0.01)
    
    print(total)
    
env.render(close=True)


[2017-03-20 10:41:45,516] Making new env: LunarLander-v2


243.797470956
-32.8488014114
219.353111093
30.568509689
176.156562556
-38.8324403062
114.753748804
218.049416314
239.012542657
-21.1754624372
204.824257629
221.1510406
233.165809066
190.769837155
226.111052744
232.375507618
246.158753518
201.513484877
260.02294592
202.344976616
225.880992423
-9.64755026126
-6.03692961905
234.001375633
209.467160545
87.5170965186
-28.0624531548
251.780547564
6.11093998915
234.962878443
243.457889913
217.791354451
231.393275445
242.334538746
233.551304121
201.214691061
184.507148375
176.875103386
203.221645247
206.039553326
228.494334658
183.623170261
233.926858399
233.831669087
183.859952167
220.885013382
229.316361555
247.971327615
225.934850965
250.040910851
111.452411384
214.949280811
114.053585001
255.138792085
130.970860274
223.51669256
-12.0294004791
259.521591465
245.417253086
95.523710089
201.21619177
195.542195408
261.715082604
237.888645878
216.372327063
154.560204699
234.574622526
226.822058493
230.557572324
210.545263243
-19.5257988617
245.6

In [64]:
run

21